## Explore zero rows

In [901]:
# Imports

import pandas as pd
import numpy as np

In [902]:
# Load data
file = "./data/scapegoats.csv"
df = pd.read_csv(file)


In [903]:
df.columns	

Index(['id', 'city', 'postal_code', 'district', 'province', 'price', 'subtype',
       'state_construction', 'living_area', 'terrace_area', 'garden_area',
       'rooms', 'bedrooms', 'bathrooms', 'livingroom_surface',
       'kitchen_surface', 'facades', 'has_garden', 'kitchen', 'furnished',
       'fireplace', 'swimmingpool', 'has_terrace', 'has_attic', 'has_basement',
       'construction_year', 'epc', 'heating', 'area_total'],
      dtype='object')

In [904]:
def IQR(column) -> float:
    """Compute lowerlimit and upperlimit to trim outliers via Tukey's fence"""
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    # quartiles = df[column].quantile([0.25, 0.75])
    iqr = Q3 - Q1
    lowerlimit = Q1 - 1.5 * iqr
    upperlimit = Q3 + 1.5 * iqr

    return lowerlimit, upperlimit

In [905]:
# Cleans price outliers
# Excludes biddings (per immoweb exploration min price of houses is 40k)
mask_price = df["price"] > 40000
df = df[mask_price]

# Cleans the price outliers via IQR function
lowerlimit, upperlimit = IQR(df["price"])
df = df[df["price"].between(lowerlimit, upperlimit)]


# Cleans total_area outliers
# Excludes small properties that are garages under wrong input on immoweb
mask_total_area = df["area_total"] > 50
df = df[mask_total_area]

# Excludes per IQR
lowerlimit_tarea, upperlimit_tarea = IQR(df["area_total"])
df = df[df["area_total"].between(lowerlimit_tarea, upperlimit_tarea)]

# Cleaning living_area outliers
# Excludes again garages and wrong inputs that fall under living_area
mask_living = df["living_area"] > 50
df = df[mask_living]

# Exclude outliers per IQR
lowerlimit_larea, upperlimit_larea = IQR(df["living_area"])
df = df[df["living_area"].between(lowerlimit_larea, upperlimit_larea)]

In [906]:
# Ensure the province names are in the same format
df["province"] = df["province"].str.lower()

In [907]:
df.replace("", np.nan, inplace=True)

# SELECT ONLY HOUSES
mask_houses = df["subtype"] == "HOUSE"
df= df[mask_houses]

# DROP columns that don't matter for analysis
# Heating is too incomplete. It has only gas, doesn't shoe electrical for example
df.drop(["id","subtype", "heating"], axis=1, inplace=True)
# Better location predictior is district, other locations are too many or too little
df.drop(["city", "postal_code", "province"], axis=1, inplace=True)
# has_terrace has less missing values than area, same as has_garden(to think)
df.drop(["terrace_area", "garden_area"], axis=1, inplace=True)
# Rooms (72% missing) is covered by bathrooms and bedrooms numbers which are more complete 
df.drop(["rooms"], axis=1, inplace=True)
# Furnished, swimmingpool, fireplace are almost all 0. Too uneven distribution for categorical data.
# And have very low correlation with the price
df.drop(['furnished','fireplace', 'swimmingpool'], axis=1, inplace=True)

# DROP rows where bathroom or bedroom is zero or NaN, because every house needs at least one
df = df[df.bedrooms != 0]
df = df[df.bathrooms != 0]
df.dropna(subset = ['bedrooms', 'bathrooms'], inplace= True)

# Drop houses with more than 4 facades, because thats weird
facade_mask = df["facades"] <=4
df = df[facade_mask]

# Selecting only columns with EPC correctly inputed
mask_epc = df["epc"].isin(["A++", "A+", "A", "B", "C", "D", "E", "F", "G"])
df = df[mask_epc]

# Group EPCS A
group_epc = {"A++": "As", "A+": "As", "A": "As"}
df = df.replace({"epc": group_epc})

# Group state of construction:

# Drop rows where kitchen or livingroom area is bigger than living area
mask_kitchen = df["kitchen_surface"] > df["living_area"]
mask_living = df["livingroom_surface"] > df["living_area"]
df = df[~mask_kitchen]
df = df[~mask_living]

# Substitute living area, terrace area and kitchen surface that are too small for NaN
df.loc[df.livingroom_surface <5 , 'livingroom_surface'] = np.nan
df.loc[df.kitchen_surface <5 , 'kitchen_surface'] = np.nan

# substitute NaN for 0 in some columns assuming no imput means it doesn't have
df["has_terrace"] = df["has_terrace"].fillna(0)
df["has_basement"] = df["has_basement"].fillna(0)
df["has_attic"] = df["has_attic"].fillna(0)
df["has_garden"] = df["has_garden"].fillna(0)




C:\Users\t_dei\AppData\Local\Temp\ipykernel_8608\2600542529.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~mask_living]


In [908]:
# Seeing missing values
percent_missing = df.isnull().sum()/ len(df)
percent_missing

district              0.000000
price                 0.000000
state_construction    0.196663
living_area           0.000000
bedrooms              0.000000
bathrooms             0.000000
livingroom_surface    0.465212
kitchen_surface       0.507151
facades               0.000000
has_garden            0.000000
kitchen               0.000000
has_terrace           0.000000
has_attic             0.000000
has_basement          0.000000
construction_year     0.367402
epc                   0.000000
area_total            0.000000
dtype: float64

In [909]:
# Measure 0 per column
print(df[df == 0].count(axis=0)/len(df.index))

district              0.000000
price                 0.000000
state_construction    0.000000
living_area           0.000000
bedrooms              0.000000
bathrooms             0.000000
livingroom_surface    0.000000
kitchen_surface       0.000000
facades               0.000000
has_garden            0.540897
kitchen               0.359505
has_terrace           0.380513
has_attic             0.648093
has_basement          0.503576
construction_year     0.000000
epc                   0.000000
area_total            0.000000
dtype: float64


In [910]:
# Measure 0 per column
print(df[df == 0].count(axis=0)/len(df.index))

district              0.000000
price                 0.000000
state_construction    0.000000
living_area           0.000000
bedrooms              0.000000
bathrooms             0.000000
livingroom_surface    0.000000
kitchen_surface       0.000000
facades               0.000000
has_garden            0.540897
kitchen               0.359505
has_terrace           0.380513
has_attic             0.648093
has_basement          0.503576
construction_year     0.000000
epc                   0.000000
area_total            0.000000
dtype: float64


In [911]:
# Force dtypes that make sense

convert_dict = {'bathrooms': 'Int64',
                "facades": 'Int64',
                'has_garden': 'Int64',
                "has_terrace": 'Int64',
                'has_attic': 'Int64',
                "has_basement": 'Int64',
                "construction_year": 'Int64'
                }
 
df = df.astype(convert_dict)

df.dtypes

district               object
price                 float64
state_construction     object
living_area           float64
bedrooms                int64
bathrooms               Int64
livingroom_surface    float64
kitchen_surface       float64
facades                 Int64
has_garden              Int64
kitchen                 int64
has_terrace             Int64
has_attic               Int64
has_basement            Int64
construction_year       Int64
epc                    object
area_total              int64
dtype: object

In [912]:
df.describe()

,price,living_area,bedrooms,bathrooms,livingroom_surface,kitchen_surface,facades,has_garden,kitchen,has_terrace,has_attic,has_basement,construction_year,area_total
count,13424.000000,13424.000000,13424.000000,13424.0,7179.000000,6616.000000,13424.0,13424.0,13424.000000,13424.0,13424.0,13424.0,8492.0,13424.000000
mean,324784.366135,169.952771,3.278829,1.215733,31.470957,14.328295,2.872169,0.459103,0.640495,0.619487,0.351907,0.496424,1959.025318,494.689139
std,143244.108566,53.354964,0.972747,0.513343,12.116256,6.931028,0.810727,0.498343,0.479873,0.485531,0.477583,0.500006,38.904561,400.742682
min,42500.000000,51.000000,1.000000,1.0,6.000000,5.000000,1.0,0.0,0.000000,0.0,0.0,0.0,1764.0,51.000000
25%,220000.000000,131.000000,3.000000,1.0,24.000000,10.000000,2.0,0.0,0.000000,0.0,0.0,0.0,1933.0,190.000000
50%,299000.000000,162.000000,3.000000,1.0,30.000000,13.000000,3.0,0.0,1.000000,1.0,0.0,0.0,1960.0,360.000000
75%,399000.000000,200.000000,4.000000,1.0,37.000000,17.000000,4.0,1.0,1.000000,1.0,1.0,1.0,1984.0,698.000000
max,890000.000000,330.000000,9.000000,15.0,186.000000,146.000000,4.0,1.0,1.000000,1.0,1.0,1.0,2025.0,1897.000000


In [913]:
df = df.sort_values(by=['price'])
df = df.reset_index(drop=True)
df

,district,price,state_construction,living_area,bedrooms,bathrooms,livingroom_surface,kitchen_surface,facades,has_garden,kitchen,has_terrace,has_attic,has_basement,construction_year,epc,area_total
0,Mons,42500.0,TO_RESTORE,175.0,4,2,NaN,NaN,3,0,1,0,1,1,<NA>,F,127
1,Mons,45000.0,TO_RESTORE,121.0,3,1,NaN,NaN,3,0,0,1,0,1,<NA>,G,180
2,Charleroi,45000.0,NaN,150.0,3,1,35.0,NaN,3,0,1,1,0,0,<NA>,E,297
3,Mons,49000.0,TO_RENOVATE,65.0,1,1,NaN,11.0,4,0,0,0,1,1,<NA>,G,916
4,Philippeville,49000.0,TO_RESTORE,99.0,4,1,30.0,NaN,2,0,1,1,1,1,<NA>,G,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13419,Brussels,880000.0,GOOD,250.0,5,2,34.0,9.0,3,1,1,1,1,1,1961,F,296
13420,Antwerp,885000.0,NaN,193.0,4,2,NaN,NaN,3,0,1,0,0,1,1902,D,604
13421,Gent,890000.0,AS_NEW,223.0,4,1,NaN,NaN,2,0,0,0,0,0,<NA>,B,100
13422,Brussels,890000.0,GOOD,264.0,5,2,20.0,26.0,2,1,0,1,1,1,1922,G,240
